In [1]:
from glob import glob
from tqdm import tqdm
import pandas as pd
import tensorflow.compat.v1 as tf
tf.logging.set_verbosity(tf.logging.ERROR)  # Hide TF deprecation messages
import matplotlib.pyplot as plt
import trimesh
import random
import numpy as np
import pickle
import modules
import data_utils

2023-07-17 23:26:30.787185: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
import datetime
from tensorflow import summary as s
log_dir = "logs/text/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# summary_writer = s.FileWriter(log_dir)

In [3]:
BATCH_SIZE=2

In [4]:
chair_meshes_paths = list(glob("chairs_ngon/*.obj"))
# chair_meshes_paths = [["Chair", path] for path in chair_meshes_paths]
tables_meshes_paths = list(glob("tables_ngon/*.obj"))
# tables_meshes_paths = [["Table", path] for path in tables_meshes_paths]
# chair_meshes_paths.extend(tables_meshes_paths)
# paths = chair_meshes_paths.copy()
# random.shuffle(paths)

In [5]:
with open("chairs_split_dict.pickle", 'rb') as f:
    chairs_split_dict = pickle.load(f)

In [6]:
with open("tables_split_dict.pickle", 'rb') as f:
    tables_split_dict = pickle.load(f)

In [7]:
chairs_train = []
chairs_val = []
chairs_test = []
for c in chair_meshes_paths: 
    try:
        split = chairs_split_dict[c.split("/")[-1].replace(".obj", "")]
    except KeyError:
        print(c.split("/")[-1].replace(".obj", ""))
        continue
    if split =='train':
        chairs_train.append(c)
    elif split =='val':
        chairs_val.append(c)
    else:
        chairs_test.append(c)
print(len(chairs_train))
print(len(chairs_val))
print(len(chairs_test))

d6075b23895c7d0880e85c92fa2351f7
248e014f31771b31d3ddfaaa242f81a1
a27b9f94d052176af27383fc5a5a711c
b0fdc43b0b4e990719abd93be2967664
1ff1912cc74a76c3b2152dcc3ff6a477
f4a0d0169abf306a754ec1f2eef2c6cc
b625936e3db6af6318df1fa50d2b64c
b6305089d351329681dff5c2e57ad46e
a680830f8b76c1bbe929777b2f481029
1512e3c41de7a461e10a48f2bbb9bef4
a1dbdff3da83425f3eec26c23f5bc80b
4afbcdeba648df2e19fb4103277a6b93
619a795a84e2566ac22e965981351403
9faefdf6814aaa975510d59f3ab1ed64
13d3462293023fe71f530727405d60cf
d851b8a016cf114c742f75bc7df727ae
3319a9b24416ecea7ff6ace05b36a5
122a480cfcdd742650c626aa72455dae
b233a919f5d2f6ac2ad490d4d7fae486
702cebffa33a19f019f079d1b712f46f
8967e65c1541d1874aa7f42ef07f614e
374fe2584abb594da094848ea4d06501
de2bb45b7e200d6916d102513d0383c0
38fed916a35dd43d7c8c503b4b873379
c73d63f9080e84a91b17743c18fb63dc
bf01483d8b58f0819767624530e7fce3
29f890e465741b7ef8cb9d3fa2bcdc0
32563c1b8b31667bb6695fcbbfeb161b
263f629d385be2b1601104cd2d998272
b5b77de7a8a05ab4f09df371fae9d63d
3dadf67ebe6c29

In [8]:
tables_train = []
tables_val = []
tables_test = []
for c in tables_meshes_paths: 
    try:
        split = tables_split_dict[c.split("/")[-1].replace(".obj", "")]
    except KeyError:
        print(c.split("/")[-1].replace(".obj", ""))
        continue
    if split =='train':
        tables_train.append(c)
    elif split =='val':
        tables_val.append(c)
    else:
        tables_test.append(c)
print(len(tables_train))
print(len(tables_val))
print(len(tables_test))

7c770e38383d59be279e896561802d26
90a2174624d8b62da53326e496617646
74983e99e7606eb114708467db3d00e2
eeb6784f5812bcebb281680297eb79f2
d6580bbf1220d39ede270fc3c23d78b
4e3d100672af00842dc02296076d8ee0
3b7fc97192e483ebb0bf045ee98272fc
812665596d6c13b1adeb1694faeea26
ba4ffdf3eed29dea9f8c3d2002c77ddb
702cebffa33a19f019f079d1b712f46f
5a6e7c4fb53f27925d3bb412062af8d4
4b6276df295b3967601104cd2d998272
6550c93bce51b9b4719088c8e42c6ab
244955211c2f3b94b821b834431f4e0e
67bee18dec77f305e4af5827e8177766
35e62b9c23928bc57f5df2bc47846a54
683ddfb4cf9c8f2883f21c04f5e0acd2
ecd56b7b86c4d94ad1f278e1513f3b7
4d24a0d4ed77db9f4b3c42e318f3affc
cca18c7f8636606f51f77a6d7299806
8b8152865839a79719fb4103277a6b93
80b20fdbf1183f37b7491f8d3451cded
8d4693c04d684e61a3430e4fb40c91fe
79fba03fadd8c00ef10b04ee8b1c6914
537a02eff282b33df58bc14da5676759
27fd962813feeb26be25fbd47cafc694
2fc468b98c0cd227c8830fa1ef6e3e71
d3b644f9eb32ec9cbe1fb1bc0addfcca
4d1d28d15b19f9101e0e41d96d3d0a78
3d95c4ebb88f94b3d2f054448bd437d6
f39401db6cc45a3

In [9]:
chairs_train.extend(tables_train)
train_paths = chairs_train.copy()
random.shuffle(train_paths)
# train_paths = train_paths[:10]

In [10]:
TRAIN_SIZE = len(train_paths)

In [11]:
chairs_val.extend(tables_val)
val_paths = chairs_val.copy()
random.shuffle(val_paths)

In [12]:
VAL_SIZE = len(val_paths)

In [13]:
captions = pd.read_csv("captions_tablechair.csv").dropna()

In [14]:
captions

,id,modelId,description,category,topLevelSynsetId,subSynsetId
0,118462,e702f89ce87a0b6579368d1198f406e7,the table is round and has 3 legs. the table i...,Table,4379243,4379243
1,118461,b654fef8eb98e99d65ba78ad9601cf1b,"the chair is made of plastic and has 4 legs, i...",Chair,3001627,4331277
2,118460,df5b55da209637624b3c42e318f3affc,It is narrow console table.It is made from ply...,Table,4379243,3092883
3,118459,791c14d53bd565f56ba14bfd91a75020,A simple chair with wooden back rest and seati...,Chair,3001627,20000020
4,118458,6730f663d0e012506f525e79a05bbcb3,It is simple chiar with comfortable back rest ...,Chair,3001627,20000020
...,...,...,...,...,...,...
75355,23226,10fbf670bb017ec94ebb593f0d0b4581,A tall table used for decoration,Table,4379243,3092883
75356,23213,bae518da1e709e83596f70d1d7edd4bc,Confortable and adjustable chair probably made...,Chair,3001627,4373704
75357,23185,bae518da1e709e83596f70d1d7edd4bc,The chair is unique and adjustable. There is ...,Chair,3001627,4373704
75358,23159,1ef6c2b9d413fb7c681404257d94ad9,Wooden table with four legs and a centerpiece ...,Table,4379243,4379243


In [15]:
max_length = 0
for c in captions['description'].values:
    cur = len(c.split(" "))
    if cur>max_length:
        max_length =cur
max_length

140

In [16]:
train_captions=[]
for index, row in captions.iterrows():
    try:
        if row["category"]=="Table":
            if tables_split_dict[row["modelId"]]=='train':
                train_captions.append(row['description'])
        if row["category"]=="Chair":
            if chairs_split_dict[row["modelId"]]=='train':
                train_captions.append(row['description'])
    except KeyError:
        continue
print(len(train_captions))

52499


In [17]:
from tensorflow.keras.preprocessing.text import Tokenizer
tk = Tokenizer()
tk.fit_on_texts(train_captions)

In [18]:
def text2shape(paths, captions, tokenizer):
    for path in paths:
        # with open(path, 'rb') as obj_file:
        mesh_dict = data_utils.load_process_mesh(path)
#         mesh_dict['class_label'] = 18 if cls=="Chair" else 49
        if len(mesh_dict['faces'])>2600:
            continue
        try:
            text = captions[captions["modelId"]==path.split("/")[-1].replace(".obj", "")].sample(n=1)["description"].values[0]
        except:
            continue
        text = text.lower().replace("the", '').replace("a", '').replace("of", '').replace("for", '').replace("and", '').replace("to", '').replace("in", '')
        text = tokenizer.texts_to_sequences([text])[0]
        mesh_dict['text_feature'] = np.pad(text, (0,max_length-len(text)))
        yield mesh_dict

In [19]:
Text2ShapeDataset = tf.data.Dataset.from_generator(
        lambda:text2shape(train_paths, captions, tk),
        output_types={
            'vertices': tf.int32, 'faces': tf.int32,
#             'class_label': tf.int32,
            'text_feature': tf.int32},
        output_shapes={
            'vertices': tf.TensorShape([None, 3]), 'faces': tf.TensorShape([None]),
#             'class_label': tf.TensorShape(()),
            'text_feature':tf.TensorShape(140)})
ex = Text2ShapeDataset.make_one_shot_iterator().get_next()
ex

{'vertices': <tf.Tensor 'IteratorGetNext:2' shape=(?, 3) dtype=int32>,
 'faces': <tf.Tensor 'IteratorGetNext:0' shape=(?,) dtype=int32>,
 'text_feature': <tf.Tensor 'IteratorGetNext:1' shape=(140,) dtype=int32>}

In [20]:
vertex_model_dataset = data_utils.make_vertex_model_dataset(
    Text2ShapeDataset, apply_random_shift=False)
vertex_model_dataset = vertex_model_dataset.repeat()
vertex_model_dataset = vertex_model_dataset.padded_batch(
    BATCH_SIZE, padded_shapes=vertex_model_dataset.output_shapes)
vertex_model_dataset = vertex_model_dataset.prefetch(1)
it = vertex_model_dataset.make_initializable_iterator()
vertex_model_batch = it.get_next()
iterator_init_op_train = it.initializer

In [21]:
Text2ShapeDatasetVal = tf.data.Dataset.from_generator(
        lambda:text2shape(val_paths, captions, tk),
        output_types={
            'vertices': tf.int32, 'faces': tf.int32,
#             'class_label': tf.int32,
            'text_feature': tf.int32},
        output_shapes={
            'vertices': tf.TensorShape([None, 3]), 'faces': tf.TensorShape([None]),
#             'class_label': tf.TensorShape(()),
            'text_feature':tf.TensorShape(140)})
vertex_model_dataset_val = data_utils.make_vertex_model_dataset(
    Text2ShapeDatasetVal, apply_random_shift=False)
vertex_model_dataset_val = vertex_model_dataset_val.repeat()
vertex_model_dataset_val = vertex_model_dataset_val.padded_batch(
    BATCH_SIZE, padded_shapes=vertex_model_dataset_val.output_shapes)
vertex_model_dataset_val = vertex_model_dataset_val.prefetch(1)
itv = vertex_model_dataset_val.make_initializable_iterator()
vertex_model_batch_val = itv.get_next()
iterator_init_op_val = itv.initializer

In [22]:
# with tf.Session() as sess:
#     ex_np = sess.run(ex)
# print(ex_np)

# # Plot the meshes
# mesh_list = []
# with tf.Session() as sess:
#     for i in range(8):
#         ex_np = sess.run(ex)
#         mesh_list.append(
#         {'vertices': data_utils.dequantize_verts(ex_np['vertices']),
#          'faces': data_utils.unflatten_faces(ex_np['faces'])})
# data_utils.plot_meshes(mesh_list, ax_lims=0.4)

In [23]:
vertex_model = modules.TextToVertexModel(
    decoder_config=dict(
      hidden_size=512,
      fc_size=2048,
      num_heads=8,
      layer_norm=True,
      num_layers=24,
      dropout_rate=0.4,
      re_zero=True,
      memory_efficient=True
      ),
    path_to_embeddings="glove.42B.300d.txt",
    embedding_dims = 300,
    quantization_bits=8,
    tokenizer=tk,
    max_num_input_verts=5000,  # number of vertices in the input mesh, if this is lower than the number of vertices in the mesh, there will be errors during training
    use_discrete_embeddings=True
)

Found 1917494 word vectors.
Converted 8256 words (1212 misses)


In [24]:
vertex_model_pred_dist = vertex_model(vertex_model_batch)
vertex_model_loss = -tf.reduce_sum(
    vertex_model_pred_dist.log_prob(vertex_model_batch['vertices_flat']) *
    vertex_model_batch['vertices_flat_mask'])
vertex_samples = vertex_model.sample(
    BATCH_SIZE, context=vertex_model_batch, max_sample_length=1000, top_p=0.95,
    recenter_verts=False, only_return_complete=False)

print(vertex_model_batch)
print(vertex_model_pred_dist)


{'vertices': <tf.Tensor 'IteratorGetNext_1:2' shape=(?, ?, 3) dtype=int32>, 'faces': <tf.Tensor 'IteratorGetNext_1:0' shape=(?, ?) dtype=int32>, 'text_feature': <tf.Tensor 'IteratorGetNext_1:1' shape=(?, 140) dtype=int32>, 'vertices_flat': <tf.Tensor 'IteratorGetNext_1:3' shape=(?, ?) dtype=int32>, 'vertices_flat_mask': <tf.Tensor 'IteratorGetNext_1:4' shape=(?, ?) dtype=float32>}
tfp.distributions.Categorical("vertex_model_2/vertex_model/create_dist/Categorical/", batch_shape=[?, ?], event_shape=[], dtype=int32)


In [25]:
vertex_model_pred_dist_val = vertex_model(vertex_model_batch_val)
vertex_model_loss_val = -tf.reduce_sum(
    vertex_model_pred_dist_val.log_prob(vertex_model_batch_val['vertices_flat']) *
    vertex_model_batch_val['vertices_flat_mask'])
vertex_samples_val = vertex_model.sample(
    BATCH_SIZE, context=vertex_model_batch_val, max_sample_length=1000, top_p=0.95,
    recenter_verts=False, only_return_complete=False)

In [26]:
# vertex_module_config=dict(
#   decoder_config=dict(
#       hidden_size=512,
#       fc_size=2048,
#       num_heads=8,
#       layer_norm=True,
#       num_layers=24,
#       dropout_rate=0.4,
#       re_zero=True,
#       memory_efficient=True
#       ),
#   quantization_bits=8,
#   class_conditional=True,
#   max_num_input_verts=5000,
#   use_discrete_embeddings=True,
#   )
# vertex_model1 = modules.VertexModel(**vertex_module_config)

In [27]:
# vertex_model_pred_dist = vertex_model(vertex_model_batch)
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     text_vars = vertex_model.variables
# print(text_vars)


In [28]:
# vertex_model1_pred_dist = vertex_model1(vertex_model_batch)
# with tf.Session() as sess:
#     orig_vars = vertex_model1.variables
# print(orig_vars)


In [29]:
# text_vars_names = []
# for v in text_vars:
# #     shape = tuple()
# #     for dim in v._variable._shape_val._dims:
# #         shape = shape+(dim._value,)
# #     text_vars_names.append(('/'.join((v._variable._name).split('/')[1:]), shape))
#     text_vars_names.append('/'.join((v._variable._name).split('/')[1:]))
# print(text_vars_names)

#'coord_embeddings/embeddings:0', 'coord_embeddings_1/embeddings:0', 'embed/embeddings:0', 'embed_zero:0', 'linear/b:0', 'linear/w:0', 'linear_1/b:0', 'linear_1/w:0'


In [30]:
# orig_vars_names = []
# for v in orig_vars:
# #     shape = tuple()
# #     for dim in v._variable._shape_val._dims:
# #         shape = shape+(dim._value,)
# #     orig_vars_names.append(('/'.join((v._variable._name).split('/')[1:]), shape))
#     orig_vars_names.append('/'.join((v._variable._name).split('/')[1:]))
# print(orig_vars_names)


In [31]:
# common_vars = set(text_vars_names) & set(orig_vars_names)
# print(common_vars)
# only_text_vars = set(text_vars_names) - common_vars
# print(only_text_vars)

In [32]:
# only_text_vars = set(text_vars_names) - common_vars
# print(only_text_vars)

In [33]:
# set(orig_vars_names) - common_vars

In [34]:
# import pickle
# with open("only_text_vars.pickle", 'wb') as out:
#     pickle.dump(only_text_vars, out)

In [35]:
with open("only_text_vars.pickle", 'rb') as f:
    only_text_vars = pickle.load(f)

text_vars = []
for var in vertex_model.variables:
    if '/'.join((var._variable._name).split('/')[1:]) in only_text_vars:
        text_vars.append(var)
text_vars=tuple(text_vars)
# print(text_vars)

In [36]:
import pickle
with open("pretrained_vars.pickle", 'rb') as f:
    common_vars = pickle.load(f)

pretrained_vars = []
for var in vertex_model.variables:
    if '/'.join((var._variable._name).split('/')[1:]) in common_vars:
        pretrained_vars.append(var)
pretrained_vars=tuple(pretrained_vars)
# print(pretrained_vars)

In [37]:
vertex_model_saver = tf.train.Saver(var_list=pretrained_vars)

In [38]:
# with tf.Session() as sess:
#   vertex_model_saver.restore(sess, "D:\\PyCharmProjects\\vertex_model\\model")

In [39]:
face_model_dataset = data_utils.make_face_model_dataset(
    Text2ShapeDataset, apply_random_shift=False)
face_model_dataset = face_model_dataset.repeat()
face_model_dataset = face_model_dataset.padded_batch(
    BATCH_SIZE, padded_shapes=face_model_dataset.output_shapes)
face_model_dataset = face_model_dataset.prefetch(1)
face_model_batch = face_model_dataset.make_one_shot_iterator().get_next()

# Create face model
face_model = modules.FaceModel(
      encoder_config=dict(
      hidden_size=512,
      fc_size=2048,
      num_heads=8,
      layer_norm=True,
      num_layers=10,
      dropout_rate=0.2,
      re_zero=True,
      memory_efficient=True,
      ),
  decoder_config=dict(
      hidden_size=512,
      fc_size=2048,
      num_heads=8,
      layer_norm=True,
      num_layers=14,
      dropout_rate=0.2,
      re_zero=True,
      memory_efficient=True,
      ),
    class_conditional=False,
    max_seq_length=8000, # number of faces in the input mesh, if this is lower than the number of vertices in the mesh, there will be errors during training
    quantization_bits=8,
    decoder_cross_attention=True,
    use_discrete_vertex_embeddings=True,
)
face_model_pred_dist = face_model(face_model_batch)
face_model_loss = -tf.reduce_sum(
    face_model_pred_dist.log_prob(face_model_batch['faces']) *
    face_model_batch['faces_mask'])
face_samples = face_model.sample(
    context=vertex_samples, max_sample_length=1000, top_p=0.95,
    only_return_complete=False)
print(face_model_batch)
print(face_model_pred_dist)
print(face_samples)

{'vertices': <tf.Tensor 'IteratorGetNext_3:4' shape=(?, ?, 3) dtype=float32>, 'faces': <tf.Tensor 'IteratorGetNext_3:0' shape=(?, ?) dtype=int64>, 'text_feature': <tf.Tensor 'IteratorGetNext_3:3' shape=(?, 140) dtype=int32>, 'num_vertices': <tf.Tensor 'IteratorGetNext_3:2' shape=(?,) dtype=int32>, 'vertices_mask': <tf.Tensor 'IteratorGetNext_3:5' shape=(?, ?) dtype=float32>, 'faces_mask': <tf.Tensor 'IteratorGetNext_3:1' shape=(?, ?) dtype=float32>}
tfp.distributions.Categorical("face_model_1/face_model/create_dist/Categorical/", batch_shape=[?, ?], event_shape=[], dtype=int32)
{'context': {'completed': <tf.Tensor 'Any:0' shape=(?,) dtype=bool>, 'vertices': <tf.Tensor 'mul_4:0' shape=(?, 1000, 3) dtype=float32>, 'num_vertices': <tf.Tensor 'floordiv:0' shape=(?,) dtype=int32>, 'vertices_mask': <tf.Tensor 'Cast_2:0' shape=(?, 1000) dtype=float32>}, 'completed': <tf.Tensor 'Any_2:0' shape=(?,) dtype=bool>, 'faces': <tf.Tensor 'Pad_2:0' shape=(?, ?) dtype=int32>, 'num_face_indices': <tf.Te

In [40]:
face_model_dataset_val = data_utils.make_face_model_dataset(
    Text2ShapeDatasetVal, apply_random_shift=False)
face_model_dataset_val  = face_model_dataset_val .repeat()
face_model_dataset_val  = face_model_dataset_val .padded_batch(
    BATCH_SIZE, padded_shapes=face_model_dataset_val.output_shapes)
face_model_dataset_val = face_model_dataset_val.prefetch(1)
face_model_batch_val = face_model_dataset_val.make_one_shot_iterator().get_next()

face_samples_val = face_model.sample(
    context=vertex_samples_val, max_sample_length=1000, top_p=0.95,
    only_return_complete=False)

In [41]:
face_model_saver = tf.train.Saver(var_list=face_model.variables)

In [42]:
text_model_saver = tf.train.Saver(var_list=vertex_model.variables, keep_checkpoint_every_n_hours=1, max_to_keep=10)

In [ ]:
from tqdm import trange
import os

with tf.variable_scope("metrics"):
    metrics = {
            'loss': tf.metrics.mean(vertex_model_loss),
            }
update_metrics_op = tf.group(*[op for _, op in metrics.values()])
metric_variables = tf.get_collection(tf.GraphKeys.LOCAL_VARIABLES, scope="metrics")
metrics_init_op = tf.variables_initializer(metric_variables)

tf.summary.scalar('loss', vertex_model_loss)
summary_op = tf.summary.merge_all()



# text_model_saver = tf.train.Saver(var_list=vertex_model.variables, keep_checkpoint_every_n_hours=1, max_to_keep=10)
last_saver = tf.train.Saver(var_list=vertex_model.variables) # will keep last 5 epochs
best_saver = tf.train.Saver(var_list=vertex_model.variables, max_to_keep=2)  # only keep 1 best checkpoint (best on eval)

%matplotlib inline 
learning_rate = 5e-4
training_steps = 500
check_step_metrics = 10
check_step_samples = 100
EPOCHS = 20
optimizer = tf.train.AdamOptimizer(learning_rate)
vertex_model_optim_op = optimizer.minimize(vertex_model_loss, var_list=text_vars)
best_v_loss = float('inf')
# Training loop
global_step = tf.train.create_global_step()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    vertex_model_saver.restore(sess, "vertex_model/model")
    face_model_saver.restore(sess, 'face_model/model')
    
    train_writer = s.FileWriter(os.path.join(log_dir, 'train_summaries'), sess.graph)
    eval_writer = s.FileWriter(os.path.join(log_dir, 'eval_summaries'), sess.graph)
    
    for e in range(EPOCHS):
        print("Epoch {}/{}".format(e + 1, EPOCHS))
        num_steps = (TRAIN_SIZE + BATCH_SIZE - 1) // BATCH_SIZE
        global_step = tf.train.get_global_step()
        sess.run(iterator_init_op_train)
        sess.run(metrics_init_op)
        t = trange(num_steps)
        for i in t:
            
            #fhjghvkerl
            if i % check_step_metrics == 0:
                _, _, loss_val, summ, global_step_val = sess.run([vertex_model_optim_op, update_metrics_op, vertex_model_loss,
                                                              summary_op, global_step])
                #_, _, loss_val, global_step_val = sess.run([vertex_model_optim_op, update_metrics_op, vertex_model_loss, global_step])
#                 sess.run(vertex_model_optim_op)
#                 sess.run(update_metrics_op)
#                 loss_val = sess.run(vertex_model_loss)
#                 global_step_val = sess.run(global_step)
                train_writer.add_summary(summ, global_step_val)
            else:
                _, _, loss_val = sess.run([vertex_model_optim_op, update_metrics_op, vertex_model_loss])
            # Log the loss in the tqdm progress bar
            t.set_postfix(loss='{:05.3f}'.format(loss_val))
        metrics_values = {k: v[0] for k, v in metrics.items()}
        metrics_val = sess.run(metrics_values)
        metrics_string = " ; ".join("{}: {:05.3f}".format(k, v) for k, v in metrics_val.items())
        print("- Train metrics: " + metrics_string)
        
        last_save_path = os.path.join("text_generaization", 'last')
        last_saver.save(sess, last_save_path, global_step=e + 1)
    
        num_steps = (VAL_SIZE + BATCH_SIZE - 1) // BATCH_SIZE
        eval_metrics = metrics
        sess.run(iterator_init_op_val)
        sess.run(metrics_init_op)
        for _ in range(num_steps):
            sess.run(update_metrics_op)
        metrics_values = {k: v[0] for k, v in eval_metrics.items()}
        metrics_val = sess.run(metrics_values)
        metrics_string = " ; ".join("{}: {:05.3f}".format(k, v) for k, v in metrics_val.items())
        print("- Eval metrics: " + metrics_string)
        global_step_val = sess.run(global_step)
        for tag, val in metrics_val.items():
            summ = tf.Summary(value=[tf.Summary.Value(tag=tag, simple_value=val)])
            eval_writer.add_summary(summ, global_step_val)
        
        if metrics_val['loss']<=best_v_loss:
            best_v_loss = metrics_val['loss']
            best_save_path = os.path.join('text_generaization', 'best')
            best_save_path = best_saver.save(sess, best_save_path, global_step=e + 1)
            print("- Found new best model, saving in {}".format(best_save_path))
                
#         #SAmples
#         if e>2:
#             sess.run(iterator_init_op_val)
#             v_samples_np, f_samples_np, b_np = sess.run((vertex_samples_val, face_samples_val, vertex_model_batch_val))
#             mesh_list = []
#             for n in range(BATCH_SIZE):
#                 mesh_list.append({
#                     'vertices': v_samples_np['vertices'][n][:v_samples_np['num_vertices'][n]],
#                     'faces': data_utils.unflatten_faces(
#                         f_samples_np['faces'][n][:f_samples_np['num_face_indices'][n]])
#                     })
#             data_utils.plot_meshes(mesh_list, ax_lims=0.5)
                

2023-07-17 23:28:30.020799: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-07-17 23:28:30.075142: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1669] Found device 0 with properties: 
name: NVIDIA GeForce RTX 3090 major: 8 minor: 6 memoryClockRate(GHz): 1.74
pciBusID: 0000:65:00.0
2023-07-17 23:28:30.075180: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2023-07-17 23:28:30.347692: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2023-07-17 23:28:30.354709: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcufft.so.10
2023-07-17 23:28:30.356745: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcurand.so.10
2023-07-17 23:28:30.359521: I tensorflow/stream_executor/platform/

Epoch 1/10


  0%|                                                                                                 | 0/5 [00:00<?, ?it/s]2023-07-17 23:29:56.104306: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
/cluster/51/ala1s/deepmind-research/polygen/data_utils.py:162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(vertices), np.array(faces)
2023-07-17 23:29:58.389495: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8
100%|███████████████████████████████████████████████████████████████████████████| 5/5 [01:37<00:00, 19.53s/it, loss=291.046]


- Train metrics: loss: 571.653
- Eval metrics: loss: 562.831
- Found new best model, saving in text_generaization/best-1
Epoch 2/10


  0%|                                                                                                 | 0/5 [00:00<?, ?it/s]

In [ ]:
# for k, v in metrics.items():
#     print(v)

In [ ]:
# from tqdm import tqdm
# %matplotlib inline 
# learning_rate = 5e-4
# training_steps = 1500
# check_step_metrics = 10
# check_step_samples = 300

# # Create an optimizer an minimize the summed log probability of the mesh
# # sequences
# optimizer = tf.train.AdamOptimizer(learning_rate)
# vertex_model_optim_op = optimizer.minimize(vertex_model_loss, var_list=text_vars)
# best_v_loss = float('inf')
# # Training loop
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     vertex_model_saver.restore(sess, "vertex_model/model")
#     face_model_saver.restore(sess, 'face_model/model')
#     for i in range(training_steps):
#         if i % check_step_metrics == 0:
#             v_loss_train = sess.run(vertex_model_loss)
# #             with summary_writer:
# #                 tf.summary.scalar('vertex_loss_train', v_loss_train)
#             v_loss_val = sess.run(vertex_model_loss_val)
# #             with summary_writer:
# #                 tf.summary.scalar('vertex_loss_val', v_loss_val)
#             if v_loss_val<best_v_loss:
#                 text_model_saver.save(sess, 'text_model/', global_step=i)
#                 best_v_loss = v_loss_val
#                 print("saved best model")
        
#             print('Step {}'.format(i))
#             print('Loss vertices train {}'.format(v_loss_train))
#             print('Loss vertices val {}'.format(v_loss_val))
#         if i % check_step_samples == 0:
#             v_samples_np, f_samples_np, b_np = sess.run((vertex_samples_val, face_samples_val, vertex_model_batch_val))
#             mesh_list = []
#             for n in range(BATCH_SIZE):
#                 mesh_list.append({
#                 'vertices': v_samples_np['vertices'][n][:v_samples_np['num_vertices'][n]],
#                 'faces': data_utils.unflatten_faces(
#                     f_samples_np['faces'][n][:f_samples_np['num_face_indices'][n]])
#                 })
#             data_utils.plot_meshes(mesh_list, ax_lims=0.5)
#         sess.run(vertex_model_optim_op)